In [29]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random

from pyzipcode import ZipCodeDatabase
zipcode=ZipCodeDatabase()

%matplotlib inline

In [30]:
df = pd.read_csv('../data/Consumer_Complaints_with_Consumer_Complaint_Narratives.csv')

# Create 'df_text' for text modeling

In [31]:
def create_df_text(df):
    df_text = pd.DataFrame()  # Create empty df to fill
    
    df_text['Consumer complaint narrative'] = df['Consumer complaint narrative']
    
    cust_resp_dict ={'Closed':0,
                 'Untimely response':0,
                 'Closed with explanation':1,
                 'Closed with non-monetary relief':2,
                 'Closed with monetary relief':2}
    
    df_text['Company response to consumer'] = df['Company response to consumer'].apply(lambda x: cust_resp_dict[x])
    
    return df_text
    

In [32]:
create_df_text(df).head()

,Consumer complaint narrative,Company response to consumer
0,Received Capital One charge card offer XXXX. A...,1
1,I do n't know how they got my cell number. I t...,1
2,I 'm a longtime member of Charter One Bank/RBS...,1
3,"After looking at my credit report, I saw a col...",2
4,I received a call from a XXXX XXXX from XXXX @...,1


# Create 'df_no_text' for non-text analysis

In [33]:
def create_df_no_text(df):
    """
    Take input df and fill all missing/NaN or non-sensical data with something
    that is useful in a model.  Also create labels column 'Company response to consumer', with 3 categories.
    INPUT - dataframe
    OUTPUT - 'df_no_text' dataframe for use in modeling non-text features
    """
    df_no_text = pd.DataFrame()  # Create empty df to fill
    
    df['Product'].fillna('Not Provided', inplace=True)
    df['Sub-product'].fillna('Not Provided', inplace=True)
    df['Sub-issue'].fillna('Not Provided', inplace=True)
    df['Issue'].fillna('Not Provided', inplace=True)
    df['Consumer complaint narrative'].fillna('Not Provided', inplace=True)
    df['Company public response'].fillna('Not Provided', inplace=True)
    df['Company'].fillna('Not Provided', inplace=True)
    df['State'].fillna('Not Provided', inplace=True)
    df['ZIP code'].fillna('Not Provided', inplace=True)
    df['Tags'].fillna('Not Provided', inplace=True)
    df['Consumer consent provided?'].fillna('Not Provided', inplace=True)
    df['Submitted via'].fillna('Not Provided',inplace=True)
    df['Consumer disputed?'].fillna('Not Provided', inplace=True)
    
    column=['Product', 'Sub-product','Issue','Sub-issue', 'Company', 'Tags', 'State']
    for name in column:
        repl={}
        i=0
        for value in df[name].unique():
            repl[value] = i
            i+=1

        df[name] = df[name].apply(lambda x: repl[x])
        df_no_text[name] = df[name].astype('category')
        
    cust_resp_dict ={'Closed':0,
                 'Untimely response':0,
                 'Closed with explanation':1,
                 'Closed with non-monetary relief':2,
                 'Closed with monetary relief':2}
    
    df_no_text['Company response to consumer'] = df['Company response to consumer'].apply(lambda x: cust_resp_dict[x])

    return df_no_text



In [36]:
create_df_no_text(df).head()

,Product,Sub-product,Issue,Sub-issue,Company,Tags,State,Company response to consumer
0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,0,1,1
2,0,0,2,0,2,0,2,1
3,2,0,3,2,3,0,3,2
4,1,2,1,3,4,0,4,1


## Other functions that I didn't need to use (see them in use (in non-function form) in the EDA_Response_Prediction_txt.ipynb)

In [38]:
def fill_missing_data(df):
    """
    Take input df and fill all missing/NaN or non-sensical data with something
    that is useful
    INPUT - dataframe
    OUTPUT - dataframe
    """
    df['Product'].fillna('Not Provided', inplace=True)
    df['Sub-product'].fillna('Not Provided', inplace=True)
    df['Sub-issue'].fillna('Not Provided', inplace=True)
    df['Issue'].fillna('Not Provided', inplace=True)
    df['Consumer complaint narrative'].fillna('Not Provided', inplace=True)
    df['Company public response'].fillna('Not Provided', inplace=True)
    df['Company'].fillna('Not Provided', inplace=True)
    # df['State'].fillna('Not Provided', inplace=True)
    ## Modified by "find_state_by_zip" function
    # df['ZIP code'].fillna('Not Provided', inplace=True)
    ## Modified by "find_state_by_zip" function
    df['Tags'].fillna('Not Provided', inplace=True)
    df['Consumer consent provided?'].fillna('Not Provided', inplace=True)
    df['Submitted via'].fillna('Not Provided',inplace=True)
    df['Consumer disputed?'].fillna('Not Provided', inplace=True)

    return df

In [39]:
def date_cleaning(df):
    """
    Clean dates, transform to datetime
    """
    df['Recieved Year'] = df['Date received'].apply(lambda x: x.year)
    df['Recieved Month'] = df['Date received'].apply(lambda x: x.month)
    df['Recieved Day'] = df['Date received'].apply(lambda x: x.day)

    df['Submitted Year'] = df['Date sent to company'].apply(lambda x: x.year)
    df['Submitted Month'] = df['Date sent to company'].apply(lambda x: x.month)
    df['Submitted Day'] = df['Date sent to company'].apply(lambda x: x.day)

    return df

In [40]:
def modify_categoricals(df):
    """
    Turn categorical variables into Yes/No, True/False, or 1/0 for input into models.
    INPUT - dataframe
    OUTPUT - dataframe
    """

    # MODIFY 'Consumer consent provided?' to be T/F
    df['Consumer consent provided?'] = df['Consumer consent provided?'].apply(lambda x:'Consent not provided' if x=='Other' or x=='Consent withdrawn' or x=='Not Provided' else x)
    
    replace_consent = {'Consent provided': True, "Consent not provided": False}
    df['Consumer consent provided?'] = df['Consumer consent provided?'].apply(lambda x: replace_consent[x])

    # MODIFY 'Consumer disputed?' to be yes/no
    df['Consumer disputed?'] = df['Consumer disputed?'].apply(lambda x: 'No' if x=='Not Provided' else x)

    replace_Y_N_to_TF = {'Yes': True, 'No':False}

    df['Consumer disputed?'] = df['Consumer disputed?'].apply(lambda x: replace_Y_N_to_TF[x])

#     MODIFY 'Timely response?'
#     df['Timely response?'] = df['Timely response?'].apply(lambda x: replace[x])

    return df


In [41]:
def count_company_complaints(df):
    """
    Create count of complaints for each company, add column
    """
    count_company_complaints = df['Company'].value_counts()
    df['Count of Company Complaints'] = df['Company'].apply(lambda x: count_company_complaints[x])

    return df

In [42]:
def find_state_by_zip(df):
    zipcode = ZipCodeDatabase
    for item in df[pd.isnull(df['State']) & pd.notnull(df['ZIP code'])].index:
        try:
            df['State'][i] = str(zip[df['ZIP code'][i]].state)
        except:
            continue

    #  Fill in empties that can't be filled with pyzipcode
    df['State'].fillna('Not provided', inplace=True)
    df['ZIP code'].fillna('Not provided', inplace=True)

    return df

In [43]:
def create_numerical_features(df):
    """
    Create numerical values for columns with many different values.
    ['Product', 'Sub-product','Issue','Sub-issue','Tags', 'State']
    Count unique items (see EDA for more info) and assign number
    INPUT - dataframe
    OUTPUT - dataframe
    """
    column=['Product', 'Sub-product','Issue','Sub-issue','Tags', 'State']
    for name in column:
        repl={}
        i=0
        for value in df[name].unique():
            repl[value] = i
            i+=1

    df[name] = df[name].apply(lambda x: repl[x])
#     df_model[name] = df[name].astype('category')

    return df


In [45]:
# fill_missing_data(df)
# date_cleaning(df)
# modify_categoricals(df)
# count_company_complaints(df)
# find_state_by_zip(df)
# create_numerical_features(df)

In [46]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/24/2015,0,0,0,0,Received Capital One charge card offer XXXX. A...,Not Provided,0,0,440XX,0,Consent provided,Web,03/30/2015,Closed with explanation,Yes,No,1297939
1,03/23/2015,1,1,1,1,I do n't know how they got my cell number. I t...,Not Provided,1,1,727XX,0,Consent provided,Web,03/23/2015,Closed with explanation,Yes,No,1296593
2,03/23/2015,0,0,2,0,I 'm a longtime member of Charter One Bank/RBS...,Not Provided,2,2,482XX,0,Consent provided,Web,03/23/2015,Closed with explanation,Yes,Yes,1296693
3,03/23/2015,2,0,3,2,"After looking at my credit report, I saw a col...",Company chooses not to provide a public response,3,3,331XX,0,Consent provided,Web,03/27/2015,Closed with non-monetary relief,Yes,No,1296955
4,03/23/2015,1,2,1,3,I received a call from a XXXX XXXX from XXXX @...,Not Provided,4,4,291XX,0,Consent provided,Web,03/23/2015,Closed with explanation,Yes,No,1296727
